In [73]:
import numpy as np
import pandas as pd
import os
import glob
import superskrypt as sp
from scipy import stats

In [74]:
# DANE_MIXED_MODELS
PTH = r"C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_MM"
os.chdir(PTH)
print(os.getcwd())

C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_MM


In [75]:
def is_odd(num):
    return num % 2

In [76]:
# counts SoA and RT for all subjects and concatenates them in a DataFrame 

db_ALL = pd.DataFrame()
db_RT = pd.DataFrame()
subj_num = []
for f in glob.glob('*.txt'):
    subj_num.append(int(f.split('.')[0][1:]))
    df = pd.read_csv(f, sep='\t')
    db_SoA = sp.get_color_info(df)
    db_RT_temp = sp.get_RT(df)
    db_ALL = db_ALL.append(db_SoA, ignore_index=True)
    db_RT = db_RT.append(db_RT_temp, ignore_index=True)
    print('read {}!'.format(f))
db_ALL.index = subj_num
db_RT.index = subj_num
print('Done!')

read s1.txt!
read s11.txt!
read s2.txt!
Done!


D:\Miniconda3\envs\py3\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [77]:
for col in db_RT.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, '{}'.format(col)] = db_RT.loc[i, col]
    print('Done col: {}'.format(col))

Done col: comp_r_RT
Done col: comp_l_RT
Done col: incomp_r_RT
Done col: incomp_l_RT
Done col: neut_r_RT
Done col: neut_l_RT
Done col: comp_RT
Done col: incomp_RT
Done col: neut_RT


In [78]:
# writing means of each condition (comp_l... etc):

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, c] = np.mean(db_ALL.loc[i, c])

In [79]:
# means of conditions of both hands:

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, 'comp'] = (db_ALL.loc[i, 'comp_l']+db_ALL.loc[i, 'comp_r'])/2
        db_ALL.loc[i, 'incomp'] = (db_ALL.loc[i, 'incomp_l']+db_ALL.loc[i, 'incomp_r'])/2
        db_ALL.loc[i, 'neut'] = (db_ALL.loc[i, 'neut_l']+db_ALL.loc[i, 'neut_r'])/2

In [80]:
# adding info about the condition

for i in db_ALL.index:
    if is_odd(i):
        db_ALL.loc[i, 'condition'] = 'info'
    else:
        db_ALL.loc[i, 'condition'] = 'noinfo'

In [81]:
db_ALL = db_ALL.loc[np.sort(db_ALL.index), :]

In [82]:
db_ALL.head()

,comp_l,comp_r,incomp_l,incomp_r,neut_l,neut_r,comp_r_RT,comp_l_RT,incomp_r_RT,incomp_l_RT,neut_r_RT,neut_l_RT,comp_RT,incomp_RT,neut_RT,comp,incomp,neut,condition
1,24.66667,12.16667,13.16667,21.66667,17.33333,16,346.888889,338.439394,354.273973,363.176471,330.200000,340.873016,342.664141,358.725222,335.536508,18.416667,17.416667,16.666667,info
2,81.16667,80.66667,81.16667,79.16667,80.16667,80.33333,385.514851,416.759259,404.449438,418.880952,399.051546,430.545455,401.137055,411.665195,414.798500,80.916667,80.166667,80.250000,noinfo
11,49.83333,67.83333,46.16667,55,58.5,56.16667,350.743590,330.416667,355.028986,360.468750,356.414286,349.563380,340.580128,357.748868,352.988833,58.833333,50.583333,57.333333,info


In [86]:
db_NEW

,subj_nr,info_noinfo,comp_incomp
1,1,info,18.416667
2,2,noinfo,80.916667
3,2,noinfo,80.250000
4,2,noinfo,80.166667
11,11,info,58.833333
12,11,info,57.333333
13,11,info,50.583333


In [84]:
db_NEW = pd.DataFrame()
for row in db_ALL.index:
    for item in range(3):
        db_NEW.loc[row+item, 'subj_nr'] = row
        db_NEW.loc[row+item, 'info_noinfo'] = db_ALL.loc[row, 'condition']
    db_NEW.loc[row, 'comp_incomp'] = db_ALL.loc[row, 'comp']
    db_NEW.loc[row+1, 'comp_incomp'] = db_ALL.loc[row, 'neut']
    db_NEW.loc[row+2, 'comp_incomp'] = db_ALL.loc[row, 'incomp']
    

In [85]:
db_ALL.index

Int64Index([1, 2, 11], dtype='int64')

In [72]:
db_NEW.to_excel('BAZA_MIXED_MODELS.xlsx')